# BDSO Multi Species Correlation
Compares the source taxonomies of the BDS Ontology with the CxG integrated MoP dataset to evaluate their compatibility

In [3]:
## IMPORT depedencies 

import os
import pandas as pd
import anndata as ad
import numpy as np
import urllib.request as request
import zipfile

## Read h5ad files

Please manually download the h5ad files from https://cellxgene.cziscience.com/collections/367d95c0-0eb0-4dae-8276-9407239421ee

In [12]:
ann_data_excitory = ad.read_h5ad("h5ad-downloads/evolution-of-cellular-diversity-excitory-cellxgene.h5ad", backed="r")
print(ann_data_excitory.obs.shape)
ann_data_excitory.obs.head(4)

(24213, 34)


,nCount_RNA,nFeature_RNA,donor_id,BICCN_cluster_label,cluster_id,cluster_color,BICCN_subclass_label,subclass_id,sample_id,nCount_SCT,...,is_primary_data,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
human_AAACCCAAGTATGGCG-21L8TX_180927_001_A01,9670.0,3579,H18.30.001,Exc L5-6 FEZF2 IFNG-AS1,116,#2c815f,L5/6 NP,12,AAACCCAAGTATGGCG-21L8TX_180927_001_A01,19454.0,...,False,nucleus,glutamatergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage
human_AAACGAAGTATGAAGT-21L8TX_180927_001_A01,25409.0,5697,H18.30.001,Exc L5 THEMIS RGPD6,100,#1c657f,L6 CT,13,AAACGAAGTATGAAGT-21L8TX_180927_001_A01,21701.0,...,False,nucleus,glutamatergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage
human_AAACGCTAGTGCACCC-21L8TX_180927_001_A01,9948.0,3736,H18.30.001,Exc L3-5 RORB RPRM,91,#7baaac,L5 IT,8,AAACGCTAGTGCACCC-21L8TX_180927_001_A01,19527.0,...,False,nucleus,glutamatergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage
human_AAAGGATAGAATTCAG-21L8TX_180927_001_A01,25874.0,5876,H18.30.001,Exc L5 THEMIS VILL,85,#4ca8ac,L5 IT,8,AAAGGATAGAATTCAG-21L8TX_180927_001_A01,21767.0,...,False,nucleus,glutamatergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage


In [13]:
ann_data_inhibitory = ad.read_h5ad("h5ad-downloads/evolution-of-cellular-diversity-inhibitory-cellxgene.h5ad", backed="r")
print(ann_data_inhibitory.obs.shape)
ann_data_inhibitory.obs.head(4)

(29486, 34)


,nCount_RNA,nFeature_RNA,donor_id,BICCN_cluster_label,cluster_id,cluster_color,BICCN_subclass_label,subclass_id,sample_id,nCount_SCT,...,is_primary_data,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
human_AAACCCAAGGATTTCC-21L8TX_180927_001_A01,12053.0,3994,H18.30.001,Inh L1-2 SST CCNJL,50,#fb8d00,Sst,5,AAACCCAAGGATTTCC-21L8TX_180927_001_A01,11870.0,...,True,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage
human_AAAGAACAGGTGAGCT-21L8TX_180927_001_A01,9523.0,3526,H18.30.001,Inh L5-6 PVALB MEPE,71,#ff5d53,Pvalb,6,AAAGAACAGGTGAGCT-21L8TX_180927_001_A01,10766.0,...,True,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage
human_AAAGAACGTACAGCGA-21L8TX_180927_001_A01,9177.0,3727,H18.30.001,Inh L1-3 VIP CHRNA2,25,#9237bb,Vip,3,AAAGAACGTACAGCGA-21L8TX_180927_001_A01,10637.0,...,True,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage
human_AAATGGAGTGAGTTTC-21L8TX_180927_001_A01,10083.0,3975,H18.30.001,Inh L5-6 PVALB FAM150B,70,#d5314c,Pvalb,6,AAATGGAGTGAGTTTC-21L8TX_180927_001_A01,10785.0,...,True,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage


In [14]:
ann_data_non_neuron = ad.read_h5ad("h5ad-downloads/evolution-of-cellular-diversity-non-neurons-cellxgene.h5ad", backed="r")
print(ann_data_non_neuron.obs.shape)
ann_data_non_neuron.obs.head(4)

(10739, 34)


,nCount_RNA,nFeature_RNA,donor_id,BICCN_cluster_label,cluster_id,cluster_color,BICCN_subclass_label,subclass_id,sample_id,nCount_SCT,...,is_primary_data,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
human_AAAGGTATCGGCTGGT-21L8TX_180927_001_A01,5026.0,2228,H18.30.001,Oligo L3-6 OPALIN ENPP6,119,#1a3329,Oligo,17,AAAGGTATCGGCTGGT-21L8TX_180927_001_A01,4132.0,...,True,nucleus,oligodendrocyte,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage
human_AAGAACACAACACGAG-21L8TX_180927_001_A01,5798.0,2767,H18.30.001,Oligo L2-6 OPALIN MAP6D1,79,#387567,Oligo,17,AAGAACACAACACGAG-21L8TX_180927_001_A01,4305.0,...,True,nucleus,oligodendrocyte,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage
human_AAGCGTTCAAGGAGTC-21L8TX_180927_001_A01,1778.0,1241,H18.30.001,Micro L1-6 TYROBP CD74,128,#94AF97,Micro-PVM,20,AAGCGTTCAAGGAGTC-21L8TX_180927_001_A01,2728.0,...,True,nucleus,microglial cell,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage
human_AATGGAAAGTCGAGGT-21L8TX_180927_001_A01,4272.0,1990,H18.30.001,Oligo L2-6 OPALIN FTH1P3,120,#003a28,Oligo,17,AATGGAAAGTCGAGGT-21L8TX_180927_001_A01,3939.0,...,True,nucleus,oligodendrocyte,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage


In [15]:
joined_obs = pd.concat([ann_data_inhibitory.obs, ann_data_excitory.obs, ann_data_non_neuron.obs])
print(joined_obs.shape)
joined_obs.head(4)

(64438, 35)


,nCount_RNA,nFeature_RNA,donor_id,BICCN_cluster_label,cluster_id,cluster_color,BICCN_subclass_label,subclass_id,sample_id,nCount_SCT,...,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,integrated_snn_res.3
human_AAACCCAAGGATTTCC-21L8TX_180927_001_A01,12053.0,3994,H18.30.001,Inh L1-2 SST CCNJL,50,#fb8d00,Sst,5,AAACCCAAGGATTTCC-21L8TX_180927_001_A01,11870.0,...,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage,NaN
human_AAAGAACAGGTGAGCT-21L8TX_180927_001_A01,9523.0,3526,H18.30.001,Inh L5-6 PVALB MEPE,71,#ff5d53,Pvalb,6,AAAGAACAGGTGAGCT-21L8TX_180927_001_A01,10766.0,...,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage,NaN
human_AAAGAACGTACAGCGA-21L8TX_180927_001_A01,9177.0,3727,H18.30.001,Inh L1-3 VIP CHRNA2,25,#9237bb,Vip,3,AAAGAACGTACAGCGA-21L8TX_180927_001_A01,10637.0,...,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage,NaN
human_AAATGGAGTGAGTTTC-21L8TX_180927_001_A01,10083.0,3975,H18.30.001,Inh L5-6 PVALB FAM150B,70,#d5314c,Pvalb,6,AAATGGAGTGAGTTTC-21L8TX_180927_001_A01,10785.0,...,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage,NaN


## Read cell_to_cell_set_assignments (c2c_data)

Downloads, unzips, loads and merges the cell to cell set assingments data for Mouse, Human and Marmoset:

In [22]:
## Mouse:
if not os.path.exists(os.path.join(os.getcwd(), "cell_to_cell_set_assignments_CCN202002013.csv")):
    # Download
    request.urlretrieve("https://github.com/AllenInstitute/MOp_taxonomies_ontology/blob/main/mouseMOp_CCN202002013/cell_to_cell_set_assignments_CCN202002013.zip?raw=true", "cell_to_cell_set_assignments_CCN202002013.zip")

    # Unzip
    with zipfile.ZipFile("cell_to_cell_set_assignments_CCN202002013.zip", 'r') as zip_ref:
        zip_ref.extractall(os.getcwd())
    
# Load to DataFrame
c2c_mouse_data = pd.read_csv("cell_to_cell_set_assignments_CCN202002013.csv")
c2c_mouse_data["sample_name"] = "mouse_" + c2c_mouse_data["sample_name"]
print(c2c_mouse_data.shape)

## Human:
if not os.path.exists(os.path.join(os.getcwd(), "cell_to_cell_set_assignments_CCN201912131.csv")):
    # Download
    request.urlretrieve("https://github.com/AllenInstitute/MOp_taxonomies_ontology/blob/main/humanM1_CCN201912131/cell_to_cell_set_assignments_CCN201912131.zip?raw=true", "cell_to_cell_set_assignments_CCN201912131.zip")

    # Unzip
    with zipfile.ZipFile("cell_to_cell_set_assignments_CCN201912131.zip", 'r') as zip_ref:
        zip_ref.extractall(os.getcwd())
    
# Load to DataFrame
c2c_human_data = pd.read_csv("cell_to_cell_set_assignments_CCN201912131.csv")
c2c_human_data["sample_name"] = "human_" + c2c_human_data["sample_name"]
print(c2c_human_data.shape)

## Marmoset:
if not os.path.exists(os.path.join(os.getcwd(), "cell_to_cell_set_assignments_CCN201912132.csv")):
    # Download
    request.urlretrieve("https://github.com/AllenInstitute/MOp_taxonomies_ontology/blob/main/marmosetM1_CCN201912132/cell_to_cell_set_assignments_CCN201912132.zip?raw=true", "cell_to_cell_set_assignments_CCN201912132.zip")

    # Unzip
    with zipfile.ZipFile("cell_to_cell_set_assignments_CCN201912132.zip", 'r') as zip_ref:
        zip_ref.extractall(os.getcwd())
    
# Load to DataFrame
c2c_marmoset_data = pd.read_csv("cell_to_cell_set_assignments_CCN201912132.csv")
c2c_marmoset_data["sample_name"] = "marmoset_" + c2c_marmoset_data["sample_name"]
print(c2c_marmoset_data.shape)

joined_c2c = pd.concat([c2c_mouse_data, c2c_human_data, c2c_marmoset_data])
print(joined_c2c.shape)
joined_c2c.head(4)

(159738, 258)
(76533, 276)
(69279, 210)
(305550, 742)


,sample_name,CS202002013_81,CS202002013_1,CS202002013_2,CS202002013_3,CS202002013_4,CS202002013_5,CS202002013_6,CS202002013_7,CS202002013_8,...,CS201912132_200,CS201912132_201,CS201912132_202,CS201912132_203,CS201912132_204,CS201912132_205,CS201912132_206,CS201912132_207,CS201912132_208,CS201912132_209
0,mouse_pBICCNsMMrMOpRAiF003d190318_AAACCCAAGGCC...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mouse_pBICCNsMMrMOpRAiF003d190318_AAACCCAAGGCT...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mouse_pBICCNsMMrMOpRAiF003d190318_AAACCCATCTGA...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mouse_pBICCNsMMrMOpRAiF003d190318_AAACGAAAGGGC...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Report the differences between c2c_data and ann_data

List all c2cdata sample names that doesn't exist in the anndata

In [43]:
adata_keys = set(joined_obs.index.tolist())
cdata_keys = set(joined_c2c.sample_name.unique().tolist())
diff = [x for x in cdata_keys if x not in adata_keys]
intersection = len(cdata_keys) - len(diff)
print("Count of the samples that exists both in anndata and cell to cell assignments data: %2d" % (intersection))
percentage = intersection * 100 / joined_obs.shape[0]
print("Ann_data size: %2d (%3.2f%% intersection)" % (joined_obs.shape[0], percentage))
print("c2c_data size: %2d (%3.2f%% intersection)" % (len(cdata_keys), intersection * 100 / len(cdata_keys) ))

Count of the samples that exists both in anndata and cell to cell assignments data: 64350
Ann_data size: 64438 (99.86% intersection)
c2c_data size: 305550 (21.06% intersection)


In [44]:
# These are not in ann-data but in cell to cell assignments data
diff2 = [x for x in adata_keys if x not in cdata_keys]
print(len(diff2))
diff2[:5]

88


['human_AAGAACAAGAGCCGAT-35L8TX_181108_001_D01',
 'human_ACTATTCGTCTACATG-23L8TX_180927_001_C01',
 'human_CACGTTCTCCATTGCC-35L8TX_181108_001_D01',
 'human_AACAAAGTCATCTGTT-34L8TX_181108_001_C01',
 'human_ATCGGCGGTCTTAGTG-35L8TX_181108_001_D01']

Filter ann_data and c2c_data to keep  the intersection rows only

In [45]:
intersection = [x for x in cdata_keys if x in adata_keys]

anndata_filtered = joined_obs[joined_obs.index.isin(intersection)]
print(anndata_filtered.shape)
anndata_filtered.head(5)

(64350, 35)


,nCount_RNA,nFeature_RNA,donor_id,BICCN_cluster_label,cluster_id,cluster_color,BICCN_subclass_label,subclass_id,sample_id,nCount_SCT,...,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,integrated_snn_res.3
human_AAACCCAAGGATTTCC-21L8TX_180927_001_A01,12053.0,3994,H18.30.001,Inh L1-2 SST CCNJL,50,#fb8d00,Sst,5,AAACCCAAGGATTTCC-21L8TX_180927_001_A01,11870.0,...,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage,NaN
human_AAAGAACAGGTGAGCT-21L8TX_180927_001_A01,9523.0,3526,H18.30.001,Inh L5-6 PVALB MEPE,71,#ff5d53,Pvalb,6,AAAGAACAGGTGAGCT-21L8TX_180927_001_A01,10766.0,...,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage,NaN
human_AAAGAACGTACAGCGA-21L8TX_180927_001_A01,9177.0,3727,H18.30.001,Inh L1-3 VIP CHRNA2,25,#9237bb,Vip,3,AAAGAACGTACAGCGA-21L8TX_180927_001_A01,10637.0,...,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage,NaN
human_AAATGGAGTGAGTTTC-21L8TX_180927_001_A01,10083.0,3975,H18.30.001,Inh L5-6 PVALB FAM150B,70,#d5314c,Pvalb,6,AAATGGAGTGAGTTTC-21L8TX_180927_001_A01,10785.0,...,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage,NaN
human_AACAAAGCAACAAGTA-21L8TX_180927_001_A01,9956.0,3424,H18.30.001,Inh L3-5 VIP TAC3,39,#9d73c6,Vip,3,AACAAAGCAACAAGTA-21L8TX_180927_001_A01,10859.0,...,nucleus,GABAergic neuron,10x 3' v3,normal,Homo sapiens,female,primary motor cortex,unknown,60-year-old human stage,NaN


In [46]:
c2c_filtered = joined_c2c[joined_c2c.sample_name.isin(intersection)]
print(c2c_filtered.shape)
c2c_filtered.head(5)

(64350, 742)


,sample_name,CS202002013_81,CS202002013_1,CS202002013_2,CS202002013_3,CS202002013_4,CS202002013_5,CS202002013_6,CS202002013_7,CS202002013_8,...,CS201912132_200,CS201912132_201,CS201912132_202,CS201912132_203,CS201912132_204,CS201912132_205,CS201912132_206,CS201912132_207,CS201912132_208,CS201912132_209
4,mouse_pBICCNsMMrMOpRAiF003d190318_AAACGAACAGAG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,mouse_pBICCNsMMrMOpRAiF003d190318_AAAGGGCAGTTC...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,mouse_pBICCNsMMrMOpRAiF003d190318_AAAGTCCCAACC...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,mouse_pBICCNsMMrMOpRAiF003d190318_AACAAAGTCTCT...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,mouse_pBICCNsMMrMOpRAiF003d190318_AACACACCAGCT...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Compare Clusterings
Test whether clustering is the same between the two datasets

Download mouse, human and marmoset nomenclature tables to find accession ids

In [49]:
if not os.path.exists(os.path.join(os.getcwd(), "nomenclature_table_CCN202002013.csv")):
    request.urlretrieve("https://raw.githubusercontent.com/obophenotype/brain_data_standards_ontologies/master/src/dendrograms/nomenclature_table_CCN202002013.csv", "nomenclature_table_CCN202002013.csv")
nomenclature_mouse = pd.read_csv("nomenclature_table_CCN202002013.csv", index_col="cell_set_accession")
print("Mouse shape: " + str(nomenclature_mouse.shape))

if not os.path.exists(os.path.join(os.getcwd(), "nomenclature_table_CCN201912131.csv")):
    request.urlretrieve("https://raw.githubusercontent.com/AllenInstitute/MOp_taxonomies_ontology/main/humanM1_CCN201912131/nomenclature_table_CCN201912131.csv", "nomenclature_table_CCN201912131.csv")
nomenclature_human = pd.read_csv("nomenclature_table_CCN201912131.csv", index_col="cell_set_accession")
print("Human shape: " + str(nomenclature_human.shape))

if not os.path.exists(os.path.join(os.getcwd(), "nomenclature_table_CCN201912132.csv")):
    request.urlretrieve("https://raw.githubusercontent.com/AllenInstitute/MOp_taxonomies_ontology/main/marmosetM1_CCN201912132/nomenclature_table_CCN201912132.csv", "nomenclature_table_CCN201912132.csv")
nomenclature_marmoset = pd.read_csv("nomenclature_table_CCN201912132.csv", index_col="cell_set_accession")
print("Marmoset shape: " + str(nomenclature_marmoset.shape))

joined_nomenclature = pd.concat([nomenclature_mouse, nomenclature_human, nomenclature_marmoset])
print(joined_nomenclature.shape)
joined_nomenclature.head(4)

Mouse shape: (257, 16)
Human shape: (275, 16)
Marmoset shape: (209, 16)
(741, 16)


,cell_set_preferred_alias,original_label,cell_set_label,cell_set_aligned_alias,cell_set_additional_aliases,cell_set_alias_assignee,cell_set_alias_citation,cell_set_structure,cell_set_ontology_tag,species,modality,taxonomy_id,taxonomy_description,child_cell_set_accessions,cell_type_card,cell_set_color
cell_set_accession,,,,,,,,,,,,,,,,
CS202002013_1,Lamp5-like Pax6,Lamp5 Pax6,RNAseq 001,Lamp5_4,Lamp5 Pax6,Zizhen Yao|Nikolas Jorstad|Zizhen Yao,PMID:34616066|PMID:34616062|PMID:34616066,primary motor cortex,UBERON:0001384,Mouse,RNAseq,CCN202002013,Mouse MOp BICCN taxonomy using multiple RNA-Se...,NaN,Cell Type,#DDACC9
CS202002013_2,Lamp5-like Egln3_1,Lamp5 Egln3_1,RNAseq 002,Sncg_4,Alpha7 cell,Zizhen Yao|Nikolas Jorstad|Federico Scala,PMID:34616066|PMID:34616062|PMID:33184512,primary motor cortex,UBERON:0001384,Mouse,RNAseq,CCN202002013,Mouse MOp BICCN taxonomy using multiple RNA-Se...,NaN,Cell Type,#FF88AD
CS202002013_3,Lamp5 Egln3_2,Lamp5 Egln3_2,RNAseq 003,NaN,Canopy cell,Zizhen Yao|Federico Scala|Zizhen Yao,PMID:34616066|PMID:33184512|PMID:34616066,primary motor cortex,UBERON:0001384,Mouse,RNAseq,CCN202002013,Mouse MOp BICCN taxonomy using multiple RNA-Se...,NaN,Cell Type,#DD8091
CS202002013_4,Lamp5 Egln3_3,Lamp5 Egln3_3,RNAseq 004,NaN,Lamp5 Egln3_3,Zizhen Yao,PMID:34616066,primary motor cortex,UBERON:0001384,Mouse,RNAseq,CCN202002013,Mouse MOp BICCN taxonomy using multiple RNA-Se...,NaN,Cell Type,#F08E98


In [51]:
nomenclature_accessions = joined_nomenclature[joined_nomenclature["cell_type_card"] != "No"].index.unique().tolist()
len(nomenclature_accessions)

396

Create an inverted mapping (cell_set_to_cell_assignments) and check if clustering in the nomenclature is same with the clustering in the ann data.
All cells that belongs to the same cluster in the nomenclature should have the same BICCN_class/BICCN_subclass/BICCN_cluster labels.

!! We don't have BICCN_class_label in this ann_data

In [56]:
records = list()
for accession in nomenclature_accessions:
    nomenclature_type = joined_nomenclature.loc[accession].cell_type_card
    samples_in_same_cluster = c2c_filtered[c2c_filtered[accession] == 1].sample_name.unique().tolist()
    for sample in samples_in_same_cluster:
        anndata_record = anndata_filtered.loc[sample]
        data = [accession, nomenclature_type, sample, anndata_record.cell_type, anndata_record.BICCN_subclass_label, anndata_record.BICCN_cluster_label]
        records.append(data)
df = pd.DataFrame(records, columns=["accession_id", "type", "sample_name", "BICCN_class_label", "BICCN_subclass_label", "BICCN_cluster_label"])
df.head(10)

,accession_id,type,sample_name,BICCN_class_label,BICCN_subclass_label,BICCN_cluster_label
0,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF003d190318_ATCACTTTCATT...,GABAergic neuron,Lamp5,Lamp5 Pax6
1,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF004d190318_AGATCGTTCATA...,GABAergic neuron,Lamp5,Lamp5 Pax6
2,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF004d190318_GGTTGTACAGAA...,GABAergic neuron,Lamp5,Lamp5 Pax6
3,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF005d190318_TTTGATCTCTCT...,GABAergic neuron,Lamp5,Lamp5 Pax6
4,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF007d190314_CATCAAGGTATC...,GABAergic neuron,Lamp5,Lamp5 Pax6
5,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF007d190314_CCGTGAGCAGTG...,GABAergic neuron,Lamp5,Lamp5 Pax6
6,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF007d190314_CGTAAGTTCACT...,GABAergic neuron,Lamp5,Lamp5 Pax6
7,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF007d190314_CTCCAACGTGAC...,GABAergic neuron,Lamp5,Lamp5 Pax6
8,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF007d190314_GAGAGGTAGTCG...,GABAergic neuron,Lamp5,Lamp5 Pax6
9,CS202002013_1,Cell Type,mouse_pBICCNsMMrMOpRAiF007d190314_GCCATGGCATTC...,GABAergic neuron,Lamp5,Lamp5 Pax6


Save inverted mapping to file cell_set_to_cell_assignments_ms_report.csv

In [57]:
df.to_csv(os.path.join(os.getcwd(), "cell_set_to_cell_assignments_ms_report.csv"))

Report the clusters that are different in the nomenclature and anndata

In [59]:
for accession in nomenclature_accessions:
    samples_in_same_cluster = c2c_filtered[c2c_filtered[accession] == 1].sample_name.unique().tolist()
    nomenclature_record = joined_nomenclature.loc[accession]
    nomenclature_type = nomenclature_record.cell_type_card
    nomenclature_label = nomenclature_record.cell_set_preferred_alias
    corresponding_ann_data = df[df.sample_name.isin(samples_in_same_cluster)]
    BICCN_class = corresponding_ann_data.BICCN_class_label.unique().tolist()
    if nomenclature_type == "Class" and len(BICCN_class) > 1:
        print("'" + nomenclature_label + "' " + accession +  "(" + nomenclature_type + ") is represented in multiple BICCN_class:" + ", ".join(BICCN_class))
    BICCN_subclass = corresponding_ann_data.BICCN_subclass_label.unique().tolist()
    if nomenclature_type == "Subclass" and len(BICCN_subclass) > 1:
        print("'" + nomenclature_label + "' " + accession +  "(" + nomenclature_type + ") is represented in multiple BICCN_subclass:" + ", ".join(BICCN_subclass))
    BICCN_clusters = corresponding_ann_data.BICCN_cluster_label.unique().tolist()
    if nomenclature_type == "Cell Type" and len(BICCN_clusters) > 1:
        print("'" + nomenclature_label + "' " + accession +  "(" + nomenclature_type + ") is represented in multiple BICCN_clusters:" + ", ".join(BICCN_clusters))    

'L6 CT' CS202002013_197(Subclass) is represented in multiple BICCN_subclass:L5/6 NP, L6 CT
'Non-neural' CS202002013_219(Class) is represented in multiple BICCN_class:endothelial cell, leptomeningeal cell, smooth muscle cell, pericyte, microglial cell
'Sst-like' CS202002013_232(Subclass) is represented in multiple BICCN_subclass:Sst Chodl, Sst
'Non-Neuronal' CS202002013_235(Class) is represented in multiple BICCN_class:oligodendrocyte precursor cell, astrocyte, oligodendrocyte
'Non-neural' CS201912131_148(Class) is represented in multiple BICCN_class:endothelial cell, leptomeningeal cell, microglial cell
'Non-Neuronal' CS201912131_151(Class) is represented in multiple BICCN_class:oligodendrocyte, oligodendrocyte precursor cell, astrocyte
'Non-Neural' CS201912132_115(Class) is represented in multiple BICCN_class:endothelial cell, pericyte, leptomeningeal cell, microglial cell
'Non-Neuronal' CS201912132_118(Class) is represented in multiple BICCN_class:oligodendrocyte, oligodendrocyte pre